# Lasso Regression for CpG Site Analysis
### Laurence Nickel (i6257119)

Libraries used: 
* pandas (version: '1.2.4')
* re (version: '2.2.1')
* sys (version: '3.8.8')
* os (version: '3.8.8')
* time (version: '3.8.8')
* plotly.express (version: '5.13.1')
* seaborn (version: '0.11.1')
* matplotlib.pyplot (version: '3.3.4')
* sklearn.linear_model (version: '0.24.1')
* sklearn.metrics (version: '0.24.1')
* joblib (version: '1.0.1')

References:
* [1] Ranstam, J., \& Cook, J. (2018). LASSO regression. *British Journal of Surgery, 105*(10), 1348. doi: https://doi.org/10.1002/bjs.10895.
* [2] Hong, J., \& Rhee, J. (2022). Genomic Effect of DNA Methylation on Gene Expression in Colorectal Cancer. *Biology (Basel), 11*(10): 1388. doi: 10.3390/biology11101388.
* [3] Miles, J. (2005). "R-squared, Adjusted R-squared," in *Encyclopedia of Statistics in Behavioral Science - Volume 4*, eds B. S. Everitt \& D. C. Howell (Hoboken, NJ, USA: John Wiley \& Sons), 1655-1657. doi: https://doi.org/10.1002/0470013192.bsa526.

## Introduction

Within this notebook, the machine learning algorithm lasso regression is performed to predict the expression levels for the genes considering the methylation values for the CpG sites with the goal of identifying the key CpG sites of DNA methylation that affect gene expression in brain cancer, especially glioblastoma. The dataset combination that is used to achieve this is the one with the M-transformed methylation data and the log2-transformed gene expression data. This was determined to be the best performing dataset combination within the notebook 'Linear Regression for Testing the Datasets.ipynb'. 

Lasso (Least Absolute Shrinkage and Selection Operator) regression is a linear regression method that uses a regularization term to perform variable selection and shrinkage of coefficient estimates [1]. Just like linear regression, lasso regression assumes a linear relationship between the independent variables and the dependent variable, but by including polynomials, it can also effectively model non-linear relationships. A line of best fit is found to relate a dependent variable to one or more independent variables, a linear equation that minimizes the sum of squared residuals. This is presented in Equation 1 where $Y$ represents the predicted gene expression value, $\beta_0$ is the intercept (bias term), $\beta_1, \beta_2, \ldots, \beta_n$ are the coefficients corresponding to each CpG site, and $\epsilon$ denotes the error term.

<br></br>
\begin{equation}
Y = \beta_0 + \beta_1 \cdot CpG_1 + \beta_2 \cdot CpG_2 + \ldots + \beta_n \cdot CpG_n + \epsilon\tag{1}
\end{equation}
<br></br>

The method Ordinary Least Squares (OLS) can be employed to estimate the coefficients $\beta_1, \beta_2, \ldots, \beta_n$. The OLS estimator aims to reduce the squared sum of differences between the observed and predicted values for gene expression. To this OLS estimator utilized by the linear regression, lasso regularization is added. This is presented in Equation 2 where $\hat{\beta}$ represents the estimated coefficient values that minimize the sum of squared errors and $N$ the number of samples.

<br></br>
\begin{equation}
\hat{\beta} = \arg\min_{\beta} \sum_{i=1}^N (Y_i - \beta_0 - \beta_1 \cdot CpG_{1i} - \beta_2 \cdot CpG_{2i} - \ldots - \beta_n \cdot CpG_{ni})^2 + \lambda \sum_{j=1}^{n} |\beta_j|\tag{2}
\end{equation}
<br></br>

The obtain the coefficients $\beta_1, \beta_2, \ldots, \beta_n$, the optimization problem in Equation 2 can be solved. Here, the $n$ represents the number of CpG sites, and the regularization parameter $\lambda$ controls the regularization strength. The term $\lambda \sum_{j=1}^{n} |\beta_j|$ introduces a penalty on the absolute values of the coefficients. By adjusting the value of $\lambda$, lasso regression can perform feature selection by forcing some coefficients to be precisely zero. This indicates that the CpG sites corresponding to those coefficients have no significant impact on the prediction of gene expression values. 

Regarding the suitability of lasso regression for this thesis, it can be applied to predict gene expression values from methylation data as it has already been successfully performed before, showing that lasso regression is suitable for working with gene expression and methylation data [2]. Please mind that this does not mean that lasso regression is necessarily the best performing (regression) method for predicting gene expression values from methylation data, but applying the algorithm might provide us with reasonable results for our purpose of identifying the key CpG sites.

From the notebooks 'Analyzing Linear Regression Results for Distance Analysis.ipynb', 'Analyzing Lasso Regression Results for Distance Analysis.ipynb', 'Analyzing Ridge Regression Results for Distance Analysis.ipynb', and 'Analyzing Elastic Net Regression Results for Distance Analysis.ipynb' we can conclude that a distance of 250,000,000 in general performs the best (of course also realizing that there may be a separate best performing distance for each of the four machine learning algorithms applied for the Distance Analysis part. This distance, as already used within the notebook 'Machine Learning Preliminary Analysis for CpG Analysis - Pearson's Correlation Coefficient.ipynb', will be used for the remainder of the CpG Analysis part and thus also for the experiments performed within this notebook.

In addition, from the notebook 'Lasso Regression for Distance Analysis.ipynb' we have discovered that using an alpha of 0.07 proved to result in the highest median R<sup>2</sup> score from the experimented with alphas. Therefore, this is also the alpha that will be used in this notebook.

Lasso regression models are built, one for each gene, and these are evaluated by using the R-squared (R<sup>2</sup>) metric which indicates the proportion of the variance in the dependent variable that can be explained by the model [3]. Higher R<sup>2</sup> values indicate a more significant proportion of the variance in the dependent variable that can be explained by the model, with 1 being the largest possible value. This R<sup>2</sup> value is retrieved by applying 4-fold cross-validation using the training and test splits defined in the notebook 'Training and Test Set Division.ipynb' present in the 'Machine Learning Algorithms - Preprocessing' folder, which also includes the motivation behind choosing the k in k-fold cross-validation to be set equal to four, and averaging the R<sup>2</sup> value for each of the four folds. Since the goal is to identify the key CpG sites of DNA methylation that affect gene expression in brain cancer (especially glioblastoma), the lasso regression models which have a positive R<sup>2</sup> value are further analyzed to achieve this within the section 'Further Analyzing the Lasso Regression Models with Positive R<sup>2</sup> scores'. For each of these models the coefficients belonging to the CpG sites used for predicting the expression value of a gene can be retrieved. The magnitude of these coefficients represent how important the methylation values for certain CpG sites were for predicting the gene expression value. Specifically, the coefficients represent the estimated change in the gene expression level associated with a unit change in the corresponding CpG site.  

Unlike we did in the notebook 'Linear Regression for Testing the Datasets.ipynb', we will not only build models for the genes which are located on chromosome 1, which was done to reduce the computational burden, but instead we will use all of the genes present within the log2-transformed gene expression data (the file 'gene_expression_data_correlation_genes_removed.csv' which is the result of applying the notebook 'Machine Learning Preliminary Analysis for CpG Analysis - Pearson's Correlation Coefficient.ipynb' where genes were removed for which at most 1 CpG site that has a higher or equal correlation coefficient with the genes than the threshold of 0.30 was present) since this notebook (among the other machine learning algorithms notebooks within this directory) represents the main experiments of the CpG Analysis part.

### Importing libraries

Before we can start to define all the functions, we should first import some libraries that will be used throughout this notebook.

In [1]:
print("Starting the importing of the libraries...")


import pandas as pd
import re
import sys
import os
import time

# Here we first need to install the plotly library.
!pip install plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

!pip install joblib
import joblib
from joblib import Parallel, delayed


print("Finishing the installing of the libraries.")

Starting the importing of the libraries...
Finishing the installing of the libraries.


Now that all the libraries have been imported, we can verify that these libraries have been loaded into this notebook by calling the version property of the library.

In [2]:
# Retrieving the version of the libraries to verify they have been correctly loaded into this notebook.
print("The library 'pd' (pandas) has been loaded into the notebook with its version being:")
print(pd.__version__)

print("\nThe library 're' has been loaded into the notebook with its version being:")
print(re.__version__)

print("\nThe library 'sys' has been loaded into the notebook with its version being:")
print(sys.version)

print("\nThe library 'plotly' has been loaded into the notebook with its version being:")
print(plotly.__version__)

print("\nThe library 'sns' (seaborn) has been loaded into the notebook with its version being:")
print(sns.__version__)

print("\nThe library 'matplotlib' has been loaded into the notebook with its version being:")
print(matplotlib.__version__)

print("\nThe library 'sklearn' has been loaded into the notebook with its version being:")
print(sklearn.__version__)

print("\nThe library 'joblib' has been loaded into the notebook with its version being:")
print(joblib.__version__)

The library 'pd' (pandas) has been loaded into the notebook with its version being:
1.2.4

The library 're' has been loaded into the notebook with its version being:
2.2.1

The library 'sys' has been loaded into the notebook with its version being:
3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]

The library 'plotly' has been loaded into the notebook with its version being:
5.13.1

The library 'sns' (seaborn) has been loaded into the notebook with its version being:
0.11.1

The library 'matplotlib' has been loaded into the notebook with its version being:
3.3.4

The library 'sklearn' has been loaded into the notebook with its version being:
0.24.1

The library 'joblib' has been loaded into the notebook with its version being:
1.0.1


### Defining the data directories

In addition, we also need to define our data directories from which the gene expression and methylation data files and the training and test splits data files will be loaded. Please mind that these need to be changed to the desired directories to be able to work with the data directories.

In [3]:
data_directory_final_datasets = "C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/final_datasets"
data_directory_final_datasets_CpG = "C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/final_datasets/CpG Site Analysis"
data_directory_training_and_test_splits = "C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/training_and_test_splits"
data_directory_results_CpG = "C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/results/CpG Site Analysis/Lasso Regression"

## Loading Training and Test Split Data

Within this section, we can load the training and test split data files from the directory 'data_directory_training_and_test_splits' into this notebook by calling the function 'pd.read_csv()' with as a parameter the to be read file.

#### Loading the 'fold_assignments_samples.csv' file into this notebook

In [4]:
# Loading the file 'fold_assignments_samples.csv'.
fold_assignments = pd.read_csv(data_directory_training_and_test_splits + '/fold_assignments_samples.csv')

print("The 'fold_assignments' DataFrame:")
fold_assignments

The 'fold_assignments' DataFrame:


,Samples,Fold
0,TCGA-06-0125-01A-01,1
1,TCGA-06-0125-02A-11,1
2,TCGA-06-0152-02A-01,2
3,TCGA-06-0171-02A-11,1
4,TCGA-06-0190-01A-01,4
...,...,...
59,TCGA-76-4927-01A-01,1
60,TCGA-76-4928-01B-01,3
61,TCGA-76-4929-01A-01,2
62,TCGA-76-4931-01A-01,2


#### Loading the 'training_and_test_assignments_samples.csv' file into this notebook

In [5]:
# Loading the file 'training_and_test_assignments_samples.csv'.
training_and_test_assignments = pd.read_csv(data_directory_training_and_test_splits + '/training_and_test_assignments_samples.csv')

print("The 'training_and_test_assignments' DataFrame:")
training_and_test_assignments

The 'training_and_test_assignments' DataFrame:


,Samples,Split 1,Split 2,Split 3,Split 4
0,TCGA-06-0125-01A-01,TEST,TRAIN,TRAIN,TRAIN
1,TCGA-06-0125-02A-11,TEST,TRAIN,TRAIN,TRAIN
2,TCGA-06-0152-02A-01,TRAIN,TEST,TRAIN,TRAIN
3,TCGA-06-0171-02A-11,TEST,TRAIN,TRAIN,TRAIN
4,TCGA-06-0190-01A-01,TRAIN,TRAIN,TRAIN,TEST
...,...,...,...,...,...
59,TCGA-76-4927-01A-01,TEST,TRAIN,TRAIN,TRAIN
60,TCGA-76-4928-01B-01,TRAIN,TRAIN,TEST,TRAIN
61,TCGA-76-4929-01A-01,TRAIN,TEST,TRAIN,TRAIN
62,TCGA-76-4931-01A-01,TRAIN,TEST,TRAIN,TRAIN


## Loading all the Different Datasets

Within this section, the datasets present within the best performing dataset combination are loaded into this notebook:
* The M-transformed methylation data file
* The log2-transformed gene expression data file

These are present in the directories 'data_directory_final_datasets' and 'data_directory_final_datasets_CpG'. For each of the corresponding files, this can be achieved by calling the function 'pd.read_csv()' with as a parameter the to be read file.

#### Loading the 'methylation_data_M_transformed_final.csv' file into this notebook

In [6]:
# Loading the file 'methylation_data_M_transformed_final.csv'.
methylation_data_M_transformed = pd.read_csv(data_directory_final_datasets + '/methylation_data_M_transformed_final.csv')

print("The 'methylation_data_M_transformed' DataFrame:")
methylation_data_M_transformed

The 'methylation_data_M_transformed' DataFrame:


,Samples,cg00000957,cg00001349,cg00001583,cg00002837,cg00003287,cg00004121,cg00008647,cg00009292,cg00011717,...,ch.22.28920330F,ch.22.436090R,ch.22.441164F,ch.22.528917R,ch.22.569473R,ch.22.707049R,ch.22.728807R,ch.22.734399R,ch.22.772318F,ch.22.909671F
0,TCGA-06-0125-01A-01,3.132755,3.960518,-5.452737,2.348422,0.642046,0.425173,-3.568310,0.090094,4.677447,...,-4.328807,-1.604700,-5.391144,-4.299671,-3.299155,-5.006189,-4.483695,-1.512707,-4.935511,-4.526937
1,TCGA-06-0125-02A-11,3.196057,3.825019,-5.503606,1.372434,0.849407,0.629880,-3.292764,1.242929,5.700119,...,-2.854379,-1.720475,-5.169584,-4.430285,-3.100187,-4.953307,-3.404266,-1.763778,-4.931599,-3.668569
2,TCGA-06-0152-02A-01,4.057813,3.626717,1.146710,-0.208610,0.059986,0.788350,-0.941862,1.416831,5.731835,...,-4.614915,-2.221432,-5.487836,-4.947837,-2.324764,-4.632719,-4.005875,-1.983516,-4.787276,-3.809136
3,TCGA-06-0171-02A-11,4.139295,3.058785,-1.109405,-0.179801,-2.005682,0.634832,-4.202176,-0.933237,6.317184,...,-3.329587,-3.007868,-5.728251,-3.993713,-3.442227,-4.986055,-3.436608,-2.831527,-5.334004,-4.291577
4,TCGA-06-0190-01A-01,3.179215,3.408476,-4.613496,-0.233366,-0.672902,0.370326,-1.536587,0.712060,4.594485,...,-3.925038,-2.666686,-5.455511,-4.393648,-3.584784,-5.394690,-3.741805,-2.292808,-4.840999,-3.959180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,TCGA-76-4927-01A-01,3.427987,4.101014,-0.268118,-0.005975,-0.769480,0.898056,-5.020576,1.241059,5.889282,...,-4.408250,-1.532935,-3.325032,-2.215352,-3.453181,-3.190405,-3.143200,-0.768262,-2.643066,-1.345063
60,TCGA-76-4928-01B-01,2.091628,3.824918,-5.990499,0.344366,-0.717767,0.425782,-5.487022,0.370565,5.714939,...,-5.370823,-1.044793,-4.456032,-2.967873,-3.687673,-3.546754,-3.881002,-1.597628,-4.209691,-2.319970
61,TCGA-76-4929-01A-01,3.166884,3.437609,-6.054600,1.700659,-4.722319,0.514961,-5.351739,1.562834,3.495553,...,-4.206406,-1.537983,-3.914480,-2.712960,-3.349122,-2.192265,-2.538937,-1.356304,-2.419188,-1.598524
62,TCGA-76-4931-01A-01,2.464759,3.631037,3.468339,0.585791,0.666281,0.879107,-5.656808,3.381448,4.041543,...,-4.784318,-1.500444,-4.241542,-4.235881,-3.584725,-2.905236,-3.847968,-0.846516,-4.245142,-1.847410


#### Loading the 'gene_expression_data_log2_transformed_correlation_genes_removed.csv' file into this notebook

In [7]:
# Loading the file 'gene_expression_data_log2_transformed_correlation_genes_removed.csv'.
gene_expression_data_log2_transformed = pd.read_csv(data_directory_final_datasets_CpG + '/gene_expression_data_log2_transformed_correlation_genes_removed.csv')

print("The 'gene_expression_data_log2_transformed' DataFrame:")
gene_expression_data_log2_transformed

The 'gene_expression_data_log2_transformed' DataFrame:


,Samples,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000288558,ENSG00000288559,ENSG00000288573,ENSG00000288586,ENSG00000288596,ENSG00000288612,ENSG00000288658,ENSG00000288667,ENSG00000288670,ENSG00000288675
0,TCGA-06-0125-01A-01,6.621171,2.915100,2.804239,3.011692,4.164312,6.016307,3.955192,4.659828,0.985136,...,3.341360,4.047233,2.695816,1.547549,3.044482,2.704385,0.767655,0.000000,3.779134,2.368740
1,TCGA-06-0125-02A-11,6.010155,2.698863,2.048550,4.123418,4.123277,6.087189,4.578244,4.262696,1.269931,...,2.808200,3.227371,2.401084,1.382944,1.933988,1.640852,0.700617,1.291662,4.009482,1.681854
2,TCGA-06-0152-02A-01,6.631346,2.883797,2.764750,4.523010,4.343927,6.211419,3.390117,4.676335,1.245252,...,3.344246,3.576861,2.495260,1.481764,2.833011,2.196733,1.697285,0.000000,3.700562,2.107219
3,TCGA-06-0171-02A-11,5.820404,2.595169,1.913148,6.059275,5.694766,6.063341,4.459025,4.545616,2.499221,...,2.170181,2.286674,1.960734,0.931078,1.921132,2.126444,0.766722,0.000000,3.113250,0.603597
4,TCGA-06-0190-01A-01,6.351695,2.551934,2.991481,4.729645,4.695788,6.674928,4.204962,4.891721,2.695816,...,2.444137,3.047172,2.077277,0.928048,2.468505,1.875387,0.229834,0.000000,2.827270,2.018029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,TCGA-76-4927-01A-01,6.032791,2.697929,2.486508,3.600924,3.809723,6.338483,4.089176,4.714811,3.187578,...,3.681371,3.211012,1.889084,1.171783,2.123335,1.187134,1.310689,0.000000,3.179065,2.604119
60,TCGA-76-4928-01B-01,6.496410,2.381450,2.285521,4.086308,2.995231,6.425281,3.453439,4.351275,2.183042,...,3.023078,2.557386,1.857344,0.440952,2.366308,1.622790,0.516923,0.000000,2.983185,1.395556
61,TCGA-76-4929-01A-01,6.521268,3.111098,1.758175,4.619372,3.469834,4.941332,4.311561,5.589269,2.704983,...,3.427901,2.907641,2.535580,2.031395,2.443501,1.956837,2.229649,0.000000,3.455663,3.203295
62,TCGA-76-4931-01A-01,6.414766,2.932118,2.799709,3.526820,3.366028,4.879471,4.093070,4.822628,2.103296,...,3.399718,3.740863,3.189129,1.569199,3.316349,3.349351,0.928579,0.000000,3.875239,2.824299


## Lasso Regression

Within this section, the machine learning algorithm lasso regression is applied using a distance of 250,000,000 to predict the expression levels for the genes considering the methylation values for the CpG sites with the goal of identifying the key CpG sites of DNA methylation that affect gene expression in brain cancer, especially glioblastoma. The dataset combination that is used to achieve this is the one with the M-transformed methylation data and the log2-transformed gene expression data. This was determined to be the best performing dataset combination within the notebook 'Linear Regression for Testing the Datasets.ipynb'. 

Lasso regression models are built, one for each gene, and these are evaluated by using the R-squared (R<sup>2</sup>) metric which indicates the proportion of the variance in the dependent variable that can be explained by the model [3]. Higher R<sup>2</sup> values indicate a more significant proportion of the variance in the dependent variable that can be explained by the model, with 1 being the largest possible value. This R<sup>2</sup> value is retrieved by applying 4-fold cross-validation using the training and test splits defined in the notebook 'Training and Test Set Division.ipynb' present in the 'Machine Learning Algorithms - Preprocessing' folder, which also includes the motivation behind choosing the k in k-fold cross-validation to be set equal to four, and averaging the R<sup>2</sup> value for each of the four folds. Since the goal is to identify the key CpG sites of DNA methylation that affect gene expression in brain cancer (especially glioblastoma), the coefficients of the lasso regression models which have a positive R<sup>2</sup> value are stored such that they can be further analyzed in the section 'Further Analyzing the Lasso Regression Models with Positive R<sup>2</sup> scores'. The magnitude of these coefficients represent how important the methylation values for certain CpG sites were for predicting the gene expression value. Specifically, the coefficients represent the estimated change in the gene expression level associated with a unit change in the corresponding CpG site.  

Unlike we did in the notebook 'Linear Regression for Testing the Datasets.ipynb', we will not only build models for the genes which are located on chromosome 1, which was done to reduce the computational burden, but instead we will use all of the genes present within the DataFrame 'gene_expression_data_log2_transformed' since this notebook (among the other machine learning algorithms notebooks within this directory) represents the main experiments of the CpG Analysis part.

In [8]:
# Setting the distance to be equal to 250,000,000.
distance = 250000000

Next, we can run the 'Machine Learning Additional Functions.ipynb' notebook present in the folder 'Machine Learning Algorithms' which contains additional helper functions, such as retrieving the methylation data within a certain distance from a gene, for the machine learning algorithms. This notebook can be run by calling the command '%run' with as argument the notebook.

In [9]:
# Running the notebook 'Machine Learning Additional Functions.ipynb' by calling the command '%run'.
%run "..\Machine Learning Additional Functions.ipynb"

Starting the importing of the libraries...
Finishing the installing of the libraries.
The library 'pd' (pandas) has been loaded into the notebook with its version being:
1.2.4
The library 'np' (numpy) has been loaded into the notebook with its version being:
1.20.1

The library 're' has been loaded into the notebook with its version being:
2.2.1

The library 'sys' has been loaded into the notebook with its version being:
3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
The 'CpG_sites_location_data' DataFrame containing the location data of the CpG sites:
The 'genes_location_data' DataFrame containing the location data of the genes:
The 'chromosomes_length_data' DataFrame containing the lengths of the chromosomes:


#### Defining the functions needed to calculate the R<sup>2</sup> scores

Next, we can define the functions needed to calculate the R<sup>2</sup> scores. We can utilize the 'joblib' library to parallelize our code as the computations for a single gene do not influence the computations of any other gene.

In [10]:
# This function calculates the R2 scores for a single 'gene' which is present in the 'gene_expression_data_log2_transformed'
# DataFrame.
def calculate_R2_scores(gene): 
    
    # Retrieving the log2-transformed gene expression data of the current 'gene' and retrieving the M-transformed 
    # methylation data that is within a distance of 'distance' from this gene and for which each of the CpG sites has a 
    # correlation higher than the threshold of 0.3 with the gene (from the notebook 'Machine Learning Preliminary Analysis 
    # for CpG Analysis - Pearson's Correlation Coefficient.ipynb') by calling the function 
    # 'get_methylation_data_close_to_gene_and_with_higher_correlation_than_threshold()' which is present in the notebook 
    # 'Machine Learning Additional Functions.ipynb'.
    gene_expression_data_current_gene = gene_expression_data_log2_transformed[['Samples', gene]]
    methylation_data_close_to_current_gene = get_methylation_data_close_to_gene_and_with_higher_correlation_than_threshold(methylation_data_M_transformed, gene_expression_data_current_gene, distance=distance, threshold=0.30)

    # Defining a list where all the R2 scores for the current gene will be stored. Since one model is built per fold (split),
    # the list 'R2_scores_current_gene' will eventually contain 4 elements (4 R2 scores).
    R2_scores_current_gene = []
    
    # Defining a list where all the information regarding the importance of the CpG sites for each gene and split 
    # combination will be stored.
    CpG_site_importance_information = []
    
    # Looping over every column in the 'training_and_test_assignments' DataFrame such that 4-fold cross-validation is 
    # performed using the training and test sets defined within that 'training_and_test_assignments' DataFrame.
    for split in training_and_test_assignments.columns[1:]:

        # Retrieving the samples which below to the training and test set for the current split 'split'.
        selected_samples_train = training_and_test_assignments.loc[training_and_test_assignments[split] == "TRAIN", 'Samples'].tolist()
        selected_samples_test = training_and_test_assignments.loc[training_and_test_assignments[split] == "TEST", 'Samples'].tolist()

        # Retrieving the gene expression and methylation data of which the samples belong to the training set.
        gene_expression_data_current_gene_train = gene_expression_data_current_gene.loc[gene_expression_data_current_gene['Samples'].isin(selected_samples_train)].drop(columns=['Samples'])
        methylation_data_close_to_current_gene_train = methylation_data_close_to_current_gene.loc[methylation_data_close_to_current_gene['Samples'].isin(selected_samples_train)].drop(columns=['Samples'])

        # Retrieving the gene expression and methylation data of which the samples belong to the test set.
        gene_expression_data_current_gene_test = gene_expression_data_current_gene.loc[gene_expression_data_current_gene['Samples'].isin(selected_samples_test)].drop(columns=['Samples'])
        methylation_data_close_to_current_gene_test = methylation_data_close_to_current_gene.loc[methylation_data_close_to_current_gene['Samples'].isin(selected_samples_test)].drop(columns=['Samples'])
        
        # Creating a new Lasso Regression model by calling the constructor 'Lasso()' with as parameter the constant that 
        # multiplies the L1 term and calling the function 'fit()' to train the model with as X-data 
        # 'methylation_data_close_to_current_gene_train' and as Y-data 'gene_expression_data_current_gene_train'.
        model_current_gene = Lasso(alpha=0.07, max_iter=2000) 
        model_current_gene.fit(methylation_data_close_to_current_gene_train, gene_expression_data_current_gene_train)
        
        # Predicting the gene expression values based on the 'methylation_data_close_to_current_gene_test' by calling the 
        # function 'predict()'.
        gene_expression_data_current_gene_predict = model_current_gene.predict(methylation_data_close_to_current_gene_test)
        
        # Calculating the R2 score by calling the function 'r2_score()' with the actual values 
        # 'gene_expression_data_current_gene_test' and the predicted values 'gene_expression_data_current_gene_predict'.
        R2_score = r2_score(gene_expression_data_current_gene_test, gene_expression_data_current_gene_predict)
            
        # If the 'R2_score' is positive for the 'model_current_gene', the information about the coefficient is stored.
        if R2_score > 0:
            
            # Retrieving the coefficients of the 'model_current_gene' by calling the property 'coef_'.
            model_coefficients = model_current_gene.coef_ 
            
            # Defining a dictionary to which all the coefficients of the CpG sites will be stored.
            CpG_dictionary = {}
            
            # Looping over all the CpG sites and adding their coefficients to the 'CpG_dictionary'.
            for index, CpG_site in enumerate(methylation_data_close_to_current_gene_train.columns):
                print(model_coefficients)
                CpG_dictionary[CpG_site] = model_coefficients[index]
            
            # Creating a dictionary to store the 'gene', 'split' and the 'CpG_dictionary'.
            model_info = {'gene': gene, 'split': split, 'R2_score': R2_score, 'coefficients': CpG_dictionary}
            
            # Appending the 'model_info' dictionary to the 'CpG_site_importance_information' list.
            CpG_site_importance_information.append(model_info)
        
        # Adding the 'R2_score' value to the 'R2_scores_current_gene' list by calling the function 'append()'.
        R2_scores_current_gene.append(R2_score)

    return {'gene': gene, 'R2': np.mean(R2_scores_current_gene), 'CpG_sites_importance': CpG_site_importance_information}

# This function retrieves the R2 scores for the lasso regression models (one for each gene) fitted to predict the   
# 'gene_expression_data_log2_transformed' based on the 'methylation_data_M_transformed'.
def lasso_regression_with_distance():
    
    # Retrieving the genes present in the log2-transformed gene expression DataFrame.
    genes = gene_expression_data_log2_transformed.columns[1:]
    
    # Defining a list where all the R2 scores (one for each gene) will be stored such that we can later represent these
    # within a box plot to compare them with the R2 scores for the other experiments. This can be achieved by calling the
    # function 'calculate_R2_scores()' for each of the genes. Since the computations for a single gene do not influence the 
    # computations of any other gene, we can parallelize the execution of this function by calling the function 'Parallel()' 
    # from the 'joblib' library.
    R2_scores = Parallel(n_jobs=512)(delayed(calculate_R2_scores)(gene) for gene in genes)
    
    # Combining all the key-value pairs belonging to the R2 scores into a single dictionary.
    R2_scores_dictionary = {R2_score['gene']: R2_score['R2'] for R2_score in R2_scores}
    
    # Combining all the key-value pairs belonging to the CpG sites importance values into a single list.
    CpG_sites_importance_list = [R2_score['CpG_sites_importance'] for R2_score in R2_scores]
    
    return CpG_sites_importance_list, R2_scores_dictionary

### Applying Lasso Regression

Now we can apply lasso regression to the M-transformed methylation data and log2-transformed gene expression data, and as mentioned before the distance of 250,000,000 is used. We can also store the resulting R<sup>2</sup> scores and the coefficients of the positive regression models by calling the function 'to.csv()' for both of them.

In [11]:
# Creating a dictionary which will later contain the lists of R2 scores for a distance of 250,000,000.
R2_scores = {'250,000,000': []}

# Creating the DataFrame by calling the constructor 'DataFrame()' which takes as input the dictionary to be converted into a
# DataFrame.
R2_scores_250_million_df = pd.DataFrame(R2_scores)

# Adding the names of the all the genes for which the R2 score is computed to the DataFrame storing all of the R2 scores.
R2_scores_250_million_df.insert(0, 'Gene', gene_expression_data_log2_transformed.columns[1:])

print("The empty 'R2_scores_250_million_df' DataFrame:")
R2_scores_250_million_df

The empty 'R2_scores_250_million_df' DataFrame:


,Gene,"250,000,000"
0,ENSG00000000419,NaN
1,ENSG00000000457,NaN
2,ENSG00000000460,NaN
3,ENSG00000000938,NaN
4,ENSG00000000971,NaN
...,...,...
19626,ENSG00000288612,NaN
19627,ENSG00000288658,NaN
19628,ENSG00000288667,NaN
19629,ENSG00000288670,NaN


In [107]:
start = time.time()
    
# Retrieving the R2 scores for the lasso regression models (one for each gene) fitted to predict the gene expression
# values based on the methylation data and retrieving the coefficients corresponding to the CpG sites for each of the models 
# with a positive R2 score by calling the function 'lasso_regression_with_distance()'
CpG_sites_importance_list, R2_scores_dictionary = lasso_regression_with_distance()

# Adding the 'R2_scores_dictionary' to the corresponding column of the general DataFrame.
R2_scores_250_million_df['250,000,000'] = R2_scores_dictionary.values()

end = time.time()
print(f"{end-start} seconds")

# Defining where to save the resulting file and its name.
file_to_save = data_directory_results_CpG + "/R2_scores_250_million_df.csv"

# Saving the file.
if not os.path.exists(file_to_save):
    R2_scores_250_million_df.to_csv(file_to_save, index=False)
    print("The file with the path " + file_to_save + " has been created.")
else:
    print("The file with the path " + file_to_save + " already exists.")

print("The 'R2_scores_250_million_df' DataFrame:")
R2_scores_250_million_df

NameError: name 'R2_scores_200_million_df' is not defined

Using the 'R2_scores_250_million_df' DataFrame to which the list of R<sup>2</sup> scores has been added, we can now create a box plot by calling the function 'boxplot()' from the 'Seaborn' library. We can also save this plot to the directory 'data_directory_results_distance' by calling the function 'savefig()'.

In [106]:
plt.figure(figsize=(20, 12))

# Creating a boxplot for the column in the 'R2_scores_250_million_df' DataFrame, plotting them on the same axis, without 
# showing the outliers.
ax = sns.boxplot(data=R2_scores_250_million_df, showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor', fontsize=17)
ax.tick_params(axis='y', labelsize=17)

# Adding the legend, a title and the labels to the plot.
ax.set_title('The Distributions of the R2 scores of a Distance of 250,000,000 Using Lasso Regression', pad=20, fontsize=30)
ax.set_xlabel('Distance', labelpad=20, fontsize=30)
ax.set_ylabel('R2 score', labelpad=20, fontsize=30)

# Saving the plot by calling the function 'savefig()'.
file_to_save = data_directory_results_CpG + f"/R2_scores_250_million_lasso_regression.png"
plt.savefig(file_to_save, bbox_inches='tight')

# Show the plot
plt.show()

NameError: name 'R2_scores_200_million_df' is not defined

<Figure size 1440x864 with 0 Axes>

In addition, we can also display how the 'CpG_sites_importance_list' list looks like by displaying the first element of the list.

In [111]:
print("The first element of the 'CpG_sites_importance_list' list:")
CpG_sites_importance_list[0]

The first element of the 'CpG_sites_importance_list' list:


[{'gene': 'ENSG00000000419',
  'split': 'Split 1',
  'R2_score': 0.2176291617811088,
  'coefficients': {'cg00309582': 0.0012284454179835478,
   'cg00351202': -0.0015279214011486159,
   'cg00375430': 0.00021202250325022217,
   'cg00388871': -0.0003564921018569464,
   'cg00410872': -0.004330756529794866,
   'cg00417421': -0.0021256074869501285,
   'cg00480389': 0.006455330363882263,
   'cg00950958': 0.00600445892478386,
   'cg00960305': -0.005935414543841752,
   'cg01176363': -0.0010731784988516403,
   'cg01278387': -0.010932732772957261,
   'cg01369406': 0.004546341362262744,
   'cg01463540': 0.005454432591642995,
   'cg01505767': -0.00480000740091664,
   'cg01548777': 0.00187129073941472,
   'cg01614729': 0.00016814359446507346,
   'cg01699217': 0.0021269349060785868,
   'cg01729827': 0.00474794877563598,
   'cg01817521': -0.00197734958795629,
   'cg01941895': -0.0027514713206377205,
   'cg01996714': 0.0016826798361435327,
   'cg02065637': -0.0015259733056080236,
   'cg02143877': -0.00

## Further Analyzing the Lasso Regression Models with Positive R<sup>2</sup> scores

Now, the next step is to retrieve how important each of the CpG sites are for the lasso regression models with positive R<sup>2</sup> scores. This can be achieved by analyzing the coefficients for each of these models that were retrieved in the section above. There are two different ways of how we can and will compute the importance of the CpG sites using the coefficients:
* Approach 1: Adding up all the coefficients values for each CpG site and calculating the importance score for each CpG site by dividing by the number of models the CpG site is utilized in. This approach is biased towards highly expressed genes as the coefficients of the CpG sites for those highly expressed genes will be higher. These highly expressed genes, however, are considered to be quite important as these kind of genes are most likely to have an active role in biological processes.
* Approach 2: Normalizing for each of the models the coefficients such that all the coefficients for a single model add up to 1. This is done such that the coefficients of the CpG sites for highly expressed genes do not automatically result in a higher importance score. To retrieve the importance score, all the coefficients values for each CpG site are added up and divided by the number of models the CpG site is utilized in.

### Approach 1

For this approach, we add up all the coefficients values for each CpG site and calculate the importance score for each CpG site by dividing by the number of models the CpG site is utilized in. This approach is biased towards highly expressed genes as the coefficients of the CpG sites for those highly expressed genes will be higher. These highly expressed genes, however, are considered to be quite important as these kind of genes are most likely to have an active role in biological processes.

In [201]:
# Creating a DataFrame which will eventually feature a single row for each of the CpG sites that were used to determine the 
# gene expression levels in the previous section. For each CpG site three columns are present: one where all the coefficients
# across all the lasso regression models with positive R<sup>2</sup> scores are added together, one which features in how
# many of the models this CpG site appears, and one which features the importance score obtained after dividing the first by
# the second column. This can be achieved by calling the constructor 'pd.DataFrame()'.
CpG_importance_scores_approach1 = pd.DataFrame(columns=['CpG Site', 'Coefficient Summation', 'Number of Models', 'Importance Score'])

# Assigning the CpG sites present within the columns of the 'methylation_data_M_transformed' DataFrame to the column 
# 'CpG Site' of the 'CpG_importance_scores_approach1' DataFrame.
CpG_importance_scores_approach1['CpG Site'] = methylation_data_M_transformed.columns[1:]

# Replacing every NaN value with a 0 as that allows us to later perform the addition.
CpG_importance_scores_approach1 = CpG_importance_scores_approach1.fillna(0)

# Setting all the numerical columns to be of approach float by using the function 'astype()' with as argument 'float'.
CpG_importance_scores_approach1[['Coefficient Summation', 'Number of Models', 'Importance Score']] = CpG_importance_scores_approach1[['Coefficient Summation', 'Number of Models', 'Importance Score']].astype(float)

print("The 'CpG_importance_scores_approach1' DataFrame:")
CpG_importance_scores_approach1

The 'CpG_importance_scores_approach1' DataFrame:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.0,0.0,0.0
1,cg00001349,0.0,0.0,0.0
2,cg00001583,0.0,0.0,0.0
3,cg00002837,0.0,0.0,0.0
4,cg00003287,0.0,0.0,0.0
...,...,...,...,...
270873,ch.22.707049R,0.0,0.0,0.0
270874,ch.22.728807R,0.0,0.0,0.0
270875,ch.22.734399R,0.0,0.0,0.0
270876,ch.22.772318F,0.0,0.0,0.0


Next, we can fill the columns 'Coefficient Summation' and 'Number of Models' of the 'CpG_importance_scores_approach1' DataFrame by looping over all models present within the 'CpG_sites_importance_list' list and retrieving the coefficients for each of the CpG sites present within that the list of coefficients.

In [202]:
# Looping over every model present in the 'CpG_sites_importance_list' list and filling the columns 'Coefficient Summation' 
# and 'Number of Models' of the 'CpG_importance_scores_approach1' DataFrame.
for gene in CpG_sites_importance_list:
    
    for model in gene:
        
        # Retrieving the 'coefficients' for the current 'model'.
        coefficients = model['coefficients']
    
        # Adding the 'coefficients' of the current 'model' to the coefficients that are already present in the column 
        # 'Coefficient Summation' of the 'CpG_importance_scores_approach1' DataFrame. This can be achieved by calling the 
        # function 'map()'. In addition, the function 'abs()' is also called for each of the coefficient values as we are 
        # investigating the importance of the CpG sites and not the kind of relationship that is present between the gene and 
        # the CpG sites. 
        CpG_importance_scores_approach1['Coefficient Summation'] += CpG_importance_scores_approach1['CpG Site'].map(coefficients).fillna(0).abs()

        # Increasing the values in the column 'Number of Models' of the 'CpG_importance_scores_approach1' DataFrame based on
        # whether the CpG sites appear within the 'coefficients' and are thus used for determining the gene expression value.
        # Also this can be achieved by calling the 'map' function and by defining a lambda expression.
        CpG_importance_scores_approach1['Number of Models'] += CpG_importance_scores_approach1['CpG Site'].map(lambda x: x in coefficients)
    
print("The 'CpG_importance_scores_approach1' DataFrame with the columns 'Coefficient Summation' and 'Number of Models' filled in:")
CpG_importance_scores_approach1

The 'CpG_importance_scores_approach1' DataFrame with the columns 'Coefficient Summation' and 'Number of Models' filled in:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.000000,0.0,0.0
1,cg00001349,0.000522,1.0,0.0
2,cg00001583,0.004527,2.0,0.0
3,cg00002837,0.000880,2.0,0.0
4,cg00003287,0.000000,0.0,0.0
...,...,...,...,...
270873,ch.22.707049R,0.000000,0.0,0.0
270874,ch.22.728807R,0.000000,0.0,0.0
270875,ch.22.734399R,0.000000,0.0,0.0
270876,ch.22.772318F,0.000000,0.0,0.0


Having now collected all of the information for the columns 'Coefficient Summation' and 'Number of Models' of the 'CpG_importance_scores_approach1' DataFrame, we can now also calculate the values for the column 'Importance Score'. This can be achieved by dividing the 'Coefficient Summation' by the 'Number of Models'.

In [203]:
# Calculating the values for the column 'Importance Score' by dividing the values present in the column 'Coefficient 
# Summation' by the values present in the column 'Number of Models'. 
CpG_importance_scores_approach1['Importance Score'] = CpG_importance_scores_approach1['Coefficient Summation'] / CpG_importance_scores_approach1['Number of Models']

# Filling the NaN values present within the column 'Importance Score' caused by performing a division by 0 with 0 by calling 
# the function 'fillna()'.
CpG_importance_scores_approach1['Importance Score'].fillna(0, inplace=True)

print("The 'CpG_importance_scores_approach1' DataFrame with all the columns filled in:")
CpG_importance_scores_approach1

The 'CpG_importance_scores_approach1' DataFrame with all the columns filled in:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.000000,0.0,0.000000
1,cg00001349,0.000522,1.0,0.000522
2,cg00001583,0.004527,2.0,0.002263
3,cg00002837,0.000880,2.0,0.000440
4,cg00003287,0.000000,0.0,0.000000
...,...,...,...,...
270873,ch.22.707049R,0.000000,0.0,0.000000
270874,ch.22.728807R,0.000000,0.0,0.000000
270875,ch.22.734399R,0.000000,0.0,0.000000
270876,ch.22.772318F,0.000000,0.0,0.000000


In addition, we can also sort the 'CpG_importance_scores_approach1' DataFrame in descending order based on the 'Importance Score' by calling the function 'sort_values()'.

In [204]:
# Sorting the 'CpG_importance_scores_approach1' DataFrame in descending order based on the 'Importance Score' by calling the 
# function 'sort_values()'.
CpG_importance_scores_approach1_sorted = CpG_importance_scores_approach1.sort_values(by='Importance Score', ascending=False)

print("The sorted 'CpG_importance_scores_approach1_sorted' DataFrame:")
CpG_importance_scores_approach1_sorted

The sorted 'CpG_importance_scores_approach1_sorted' DataFrame:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
139463,cg21072248,0.013624,1.0,0.013624
69913,cg16815850,0.012562,1.0,0.012562
132981,cg09702096,0.011933,1.0,0.011933
260845,cg00301998,0.011927,1.0,0.011927
73128,cg21393051,0.011023,1.0,0.011023
...,...,...,...,...
96147,cg03906834,0.000000,0.0,0.000000
96148,cg03906843,0.000000,0.0,0.000000
96149,cg03907849,0.000000,0.0,0.000000
96150,cg03911034,0.000000,0.0,0.000000


Finally, we can also store the 'CpG_importance_scores_approach1_sorted' DataFrame to the local directory 'data_directory_results_CpG' by calling the function 'to_csv()'.

In [205]:
# Defining where to save the resulting file and its name.
file_to_save = data_directory_results_CpG + "/CpG_importance_scores_approach1_sorted.csv"

# Saving the file.
if not os.path.exists(file_to_save):
    CpG_importance_scores_approach1_sorted.to_csv(file_to_save, index=False)
    print("The file with the path " + file_to_save + " has been created.")
else:
    print("The file with the path " + file_to_save + " already exists.")

The file with the path C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/results/CpG Site Analysis/Linear Regression/CpG_importance_scores_approach1_sorted.csv already exists.


### Approach 2

For this approach, we normalize for each of the models the coefficients such that all the coefficients for a single model add up to 1. This is done such that the coefficients of the CpG sites for highly expressed genes do not automatically result in a higher importance score. To retrieve the importance score, all the coefficients values for each CpG site are added up and divided by the number of models the CpG site is utilized in.

In [206]:
# Creating a DataFrame which will eventually feature a single row for each of the CpG sites that were used to determine the 
# gene expression levels in the previous section. For each CpG site three columns are present: one where all the coefficients
# across all the lasso regression models with positive R<sup>2</sup> scores are added together, one which features in how
# many of the models this CpG site appears, and one which features the importance score obtained after dividing the first by
# the second column. This can be achieved by calling the constructor 'pd.DataFrame()'.
CpG_importance_scores_approach2 = pd.DataFrame(columns=['CpG Site', 'Coefficient Summation', 'Number of Models', 'Importance Score'])

# Assigning the CpG sites present within the columns of the 'methylation_data_M_transformed' DataFrame to the column 
# 'CpG Site' of the 'CpG_importance_scores_approach2' DataFrame.
CpG_importance_scores_approach2['CpG Site'] = methylation_data_M_transformed.columns[1:]

# Replacing every NaN value with a 0 as that allows us to later perform the addition.
CpG_importance_scores_approach2 = CpG_importance_scores_approach2.fillna(0)

# Setting all the numerical columns to be of approach float by using the function 'astype()' with as argument 'float'.
CpG_importance_scores_approach2[['Coefficient Summation', 'Number of Models', 'Importance Score']] = CpG_importance_scores_approach2[['Coefficient Summation', 'Number of Models', 'Importance Score']].astype(float)

print("The 'CpG_importance_scores_approach2' DataFrame:")
CpG_importance_scores_approach2

The 'CpG_importance_scores_approach2' DataFrame:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.0,0.0,0.0
1,cg00001349,0.0,0.0,0.0
2,cg00001583,0.0,0.0,0.0
3,cg00002837,0.0,0.0,0.0
4,cg00003287,0.0,0.0,0.0
...,...,...,...,...
270873,ch.22.707049R,0.0,0.0,0.0
270874,ch.22.728807R,0.0,0.0,0.0
270875,ch.22.734399R,0.0,0.0,0.0
270876,ch.22.772318F,0.0,0.0,0.0


Next, we can fill the columns 'Coefficient Summation' and 'Number of Models' of the 'CpG_importance_scores_approach2' DataFrame by looping over all models present within the 'CpG_sites_importance_list' list and retrieving the coefficients for each of the CpG sites present within that the list of coefficients. Before we fill in these columns we will first normalize the coefficients such that all the coefficients values for a single model add up to 1.

In [207]:
# Looping over every model present in the 'CpG_sites_importance_list' list and filling the columns 'Coefficient Summation' 
# and 'Number of Models' of the 'CpG_importance_scores_approach2' DataFrame.
for gene in CpG_sites_importance_list:
    
    for model in gene:
        
        # Retrieving the 'coefficients' for the current 'model'.
        coefficients = model['coefficients']
    
        # Next, we normalize the 'coefficients' such that all the coefficients values for a single model add up to 1. To achieve
        # this, first the sum of the 'coefficients' will be calculated by calling the function 'sum()'. Here, we first call the
        # function 'abs()' as we are investigating the importance of the CpG sites and not the kind of relationship that is 
        # present between the gene and the CpG sites. Next, we can calculate the new normalized values by dividing every value
        # by the 'coefficients_sum'.
        coefficients_sum = sum(abs(value) for value in coefficients.values())
        normalized_coefficients = {key: abs(value) / coefficients_sum for key, value in coefficients.items()}

        # Adding the 'normalized_coefficients' of the current 'model' to the coefficients that are already present in the column 
        # 'Coefficient Summation' of the 'CpG_importance_scores_approach2' DataFrame. This can be achieved by calling the 
        # function 'map()'. In addition, the function 'abs()' is also called for each of the coefficient values as we are 
        # investigating the importance of the CpG sites and not the kind of relationship that is present between the gene and 
        # the CpG sites. 
        CpG_importance_scores_approach2['Coefficient Summation'] += CpG_importance_scores_approach2['CpG Site'].map(normalized_coefficients).fillna(0).abs()

        # Increasing the values in the column 'Number of Models' of the 'CpG_importance_scores_approach2' DataFrame based on
        # whether the CpG sites appear within the 'normalized_coefficients' and are thus used for determining the gene 
        # expression value. Also this can be achieved by calling the 'map' function and by defining a lambda expression.
        CpG_importance_scores_approach2['Number of Models'] += CpG_importance_scores_approach2['CpG Site'].map(lambda x: x in normalized_coefficients)
    
print("The 'CpG_importance_scores_approach2' DataFrame with the columns 'Coefficient Summation' and 'Number of Models' filled in:")
CpG_importance_scores_approach2

The 'CpG_importance_scores_approach2' DataFrame with the columns 'Coefficient Summation' and 'Number of Models' filled in:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.000000,0.0,0.0
1,cg00001349,0.000270,1.0,0.0
2,cg00001583,0.002087,2.0,0.0
3,cg00002837,0.000207,2.0,0.0
4,cg00003287,0.000000,0.0,0.0
...,...,...,...,...
270873,ch.22.707049R,0.000000,0.0,0.0
270874,ch.22.728807R,0.000000,0.0,0.0
270875,ch.22.734399R,0.000000,0.0,0.0
270876,ch.22.772318F,0.000000,0.0,0.0


Having now collected all of the information for the columns 'Coefficient Summation' and 'Number of Models' of the 'CpG_importance_scores_approach2' DataFrame, we can now also calculate the values for the column 'Importance Score'. This can be achieved by dividing the 'Coefficient Summation' by the 'Number of Models'.

In [208]:
# Calculating the values for the column 'Importance Score' by dividing the values present in the column 'Coefficient 
# Summation' by the values present in the column 'Number of Models'. 
CpG_importance_scores_approach2['Importance Score'] = CpG_importance_scores_approach2['Coefficient Summation'] / CpG_importance_scores_approach2['Number of Models']

# Filling the NaN values present within the column 'Importance Score' caused by performing a division by 0 with 0 by calling 
# the function 'fillna()'.
CpG_importance_scores_approach2['Importance Score'].fillna(0, inplace=True)

print("The 'CpG_importance_scores_approach2' DataFrame with all the columns filled in:")
CpG_importance_scores_approach2

The 'CpG_importance_scores_approach2' DataFrame with all the columns filled in:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
0,cg00000957,0.000000,0.0,0.000000
1,cg00001349,0.000270,1.0,0.000270
2,cg00001583,0.002087,2.0,0.001044
3,cg00002837,0.000207,2.0,0.000103
4,cg00003287,0.000000,0.0,0.000000
...,...,...,...,...
270873,ch.22.707049R,0.000000,0.0,0.000000
270874,ch.22.728807R,0.000000,0.0,0.000000
270875,ch.22.734399R,0.000000,0.0,0.000000
270876,ch.22.772318F,0.000000,0.0,0.000000


In addition, we can also sort the 'CpG_importance_scores_approach2' DataFrame in descending order based on the 'Importance Score' by calling the function 'sort_values()'.

In [209]:
# Sorting the 'CpG_importance_scores_approach2' DataFrame in descending order based on the 'Importance Score' by calling the 
# function 'sort_values()'.
CpG_importance_scores_approach2_sorted = CpG_importance_scores_approach2.sort_values(by='Importance Score', ascending=False)

print("The sorted 'CpG_importance_scores_approach2_sorted' DataFrame:")
CpG_importance_scores_approach2_sorted

The sorted 'CpG_importance_scores_approach2_sorted' DataFrame:


,CpG Site,Coefficient Summation,Number of Models,Importance Score
260845,cg00301998,0.006660,1.0,0.006660
53478,cg01278387,0.006104,1.0,0.006104
264951,cg25184787,0.005878,1.0,0.005878
261119,cg01266916,0.005826,1.0,0.005826
139463,cg21072248,0.005742,1.0,0.005742
...,...,...,...,...
96147,cg03906834,0.000000,0.0,0.000000
96148,cg03906843,0.000000,0.0,0.000000
96149,cg03907849,0.000000,0.0,0.000000
96150,cg03911034,0.000000,0.0,0.000000


Finally, we can also store the 'CpG_importance_scores_approach2_sorted' DataFrame to the local directory 'data_directory_results_CpG' by calling the function 'to_csv()'.

In [210]:
# Defining where to save the resulting file and its name.
file_to_save = data_directory_results_CpG + "/CpG_importance_scores_approach2_sorted.csv"

# Saving the file.
if not os.path.exists(file_to_save):
    CpG_importance_scores_approach2_sorted.to_csv(file_to_save, index=False)
    print("The file with the path " + file_to_save + " has been created.")
else:
    print("The file with the path " + file_to_save + " already exists.")

The file with the path C:/Users/laure/OneDrive/Documenten/Bachelor Thesis Data/results/CpG Site Analysis/Linear Regression/CpG_importance_scores_approach2_sorted.csv already exists.
